# Respiration

In [2]:
def rr(start, end, print1, print2):
    %run configuration.ipynb
    if start != 0:
        start = start - MARGIN
    end = end + MARGIN
    resp = []
    filename = PATIENT_ID + "-MDC_RESP-62.5.csv"
    if print1 or print2:
        print(filename+':')
        print("sampling frequency is ", F," Hz")
    with open('samples/'+filename, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            resp.append(float(row[1]))

    resp_small = np.array(resp[(int)(start*62.5):(int)(end*62.5)])            
    peaks,_=find_peaks(resp_small, distance=31, height=1150, prominence=1)
    peaks_sec = (peaks + start*62.5) * 0.016
    time = np.arange(start, end, 0.016)

    if print1:
        plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT))
        plt.plot(time,resp_small)
        plt.plot(peaks_sec, resp_small[peaks], "x")
        plt.title('Respiration')
        plt.xlabel('Time [sec]')
        plt.show()

    diffs = np.diff(peaks_sec)
    peaks_sec = peaks_sec[:len(diffs)]+diffs/2
    rr = 60/diffs

    resp = []

    rr_interp_func = interp1d(peaks_sec, rr)
    time = np.arange(start + MARGIN, end - MARGIN + 1/F, 1/F)

    if print2:
        plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT))
        plt.plot(peaks_sec,rr)
        plt.plot(time,rr_interp_func(time))
        plt.title('Respiration Rate')
        plt.xlabel('Time [sec]')
        plt.ylabel('[Hz]')
        plt.legend(['data', 'resampled'], loc='best')
        plt.show()

    return time, rr_interp_func(time)
